<a href="https://colab.research.google.com/github/sornavel/Hackathons-Janatahack/blob/master/Healthcare_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

In [ ]:
import os
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
np.random.seed(0)
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
from IPython.core.display import HTML 
from IPython.display import Image
import gc
from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1FUu6ZyHdGB_ElcE4ZuwzPAh-DbQ7mAwQ'})
download.GetContentFile('train.csv')
train_df = pd.read_csv('train.csv')

download = drive.CreateFile({'id': '1vE_M60f3HJ4R1GSFl3QjHWzuwEIcVrmd'})
download.GetContentFile('Third_Health_Camp_Attended.csv')
third_df = pd.read_csv('Third_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '1hnoPLF_uC4hrpCcSFmGJ1UxLog7grgrl'})
download.GetContentFile('Second_Health_Camp_Attended.csv')
second_df = pd.read_csv('Second_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '1LMzFdyra4gObQctf5h0lb6DtUnqiLBT3'})
download.GetContentFile('First_Health_Camp_Attended.csv')
first_df = pd.read_csv('First_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '16JMb9xNz5BEMlV9dS7reh9yi8ivGHYT2'})
download.GetContentFile('Patient_Profile.csv')
patient_df = pd.read_csv('Patient_Profile.csv')

download = drive.CreateFile({'id': '16AFNM3SlDDRMpEAdq4ssddK5QwoH9S33'})
download.GetContentFile('Health_Camp_Detail.csv')
camp_df = pd.read_csv('Health_Camp_Detail.csv')

download = drive.CreateFile({'id': '1aQqKgHFJ0M5j-V34G_TZgOIi2xc6GnaA'})
download.GetContentFile('test.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
first_df.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [ ]:
second_df.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [ ]:
third_df.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [ ]:
patient_df.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [ ]:
camp_df.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [ ]:
test_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [ ]:
third_df_filtered = third_df[third_df.Number_of_stall_visited!=0]

In [ ]:
df = pd.concat([train_df,test_df],axis=0)
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
temp = pd.merge(df,first_df,how='left', on=['Patient_ID','Health_Camp_ID'])
temp.Health_Score.loc[~temp.Health_Score.isnull()] = 1
temp.Health_Score.loc[temp.Health_Score.isnull()] = 0
df = temp.rename(columns={'Health_Score':'succ_1'})

temp = pd.merge(df,second_df,how='left', on=['Patient_ID','Health_Camp_ID'])
temp['Health Score'].loc[~temp['Health Score'].isnull()] = 1
temp['Health Score'].loc[temp['Health Score'].isnull()] = 0
df = temp.rename(columns={'Health Score':'succ_2'})

temp = pd.merge(df,third_df_filtered,how='left', on=['Patient_ID','Health_Camp_ID'])
temp['Number_of_stall_visited'].loc[~temp['Number_of_stall_visited'].isnull()] = 1
temp['Number_of_stall_visited'].loc[temp['Number_of_stall_visited'].isnull()] = 0
df = temp.rename(columns={'Number_of_stall_visited':'succ_3'})
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,succ_1,Unnamed: 4,succ_2,succ_3,Last_Stall_Visited_Number
0,489652,6578,10-Sep-05,4,0,0,0,2,NaN,0.0,NaN,0.0,1.0,1.0
1,507246,6578,18-Aug-05,45,5,0,0,7,NaN,0.0,NaN,0.0,0.0,NaN
2,523729,6534,29-Apr-06,0,0,0,0,0,NaN,0.0,NaN,1.0,0.0,NaN
3,524931,6535,07-Feb-04,0,0,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
4,521364,6529,28-Feb-06,15,1,0,0,7,NaN,0.0,NaN,1.0,0.0,NaN


In [ ]:
df.drop(['Donation','Unnamed: 4','Last_Stall_Visited_Number'],axis=1,inplace=True)

In [ ]:
df['Reg_year'] = pd.DatetimeIndex(df.Registration_Date).year
df['Reg_month'] = pd.DatetimeIndex(df.Registration_Date).month
df['Reg_day'] = pd.DatetimeIndex(df.Registration_Date).day

In [ ]:
temp = pd.merge(df,patient_df,on='Patient_ID',how='left')
df=temp
temp = pd.merge(df,camp_df,on='Health_Camp_ID',how='left')
df=temp

In [ ]:
df.drop(['Patient_ID','Health_Camp_ID','Registration_Date',],axis=1,inplace=True)

In [ ]:
df.Income[df.Income=='None'] = 0
df.Education_Score[df.Education_Score=='None'] = 0
df.Age[df.Age=='None'] = 0
df.Age = df.Age.astype('int64')
df.Age[df.Age==0] = df.Age[df.Age!=0].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df['fi_year'] = pd.DatetimeIndex(df.First_Interaction).year
df['fi_month'] = pd.DatetimeIndex(df.First_Interaction).month
df['fi_day'] = pd.DatetimeIndex(df.First_Interaction).day

In [ ]:
df.drop(['First_Interaction'],axis=1,inplace=True)

In [ ]:
#temp = df.City_Type[~df.City_Type.isnull()].mode()
df.City_Type = df.City_Type.replace(np.nan,'B')
df.Employer_Category = df.Employer_Category.replace(np.nan,'Technology')

In [ ]:
temp = pd.to_datetime(df.Camp_End_Date) - pd.to_datetime(df.Camp_Start_Date)
df['camp_days'] = temp.dt.days

In [ ]:
df.drop(['Camp_Start_Date', 'Camp_End_Date'],axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
df['City_Type']= le.fit_transform(df['City_Type']) 
df['Category1']= le.fit_transform(df['Category1']) 
df['Category2']= le.fit_transform(df['Category2']) 
df['Employer_Category']= le.fit_transform(df['Employer_Category']) 


In [ ]:
#temp = df.Reg_year[~df.Reg_year.isnull()].mode() = 2006
df.Reg_year = df.Reg_year.replace(np.nan,2006)
df.Reg_month = df.Reg_month.replace(np.nan,9)
df.Reg_day = df.Reg_day.replace(np.nan,18)

In [ ]:
df['succ'] = df['succ_1'] + df['succ_2'] + df['succ_3']

In [ ]:
y_train = df['succ'][:train_df.shape[0]]
df.drop(['succ','succ_1','succ_2','succ_3'],axis=1,inplace = True)

In [ ]:
df.head()

,Var1,Var2,Var3,Var4,Var5,Reg_year,Reg_month,Reg_day,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,fi_year,fi_month,fi_day,camp_days
0,4,0,0,0,2,2005.0,9.0,10.0,0,0,0,0,0,0,48.20876,1,11,2,6,2,2004,12,6,59
1,45,5,0,0,7,2005.0,8.0,18.0,0,0,0,0,1,75,40.00000,2,7,2,6,2,2004,9,8,59
2,0,0,0,0,0,2006.0,4.0,29.0,0,0,0,0,0,0,48.20876,1,11,1,0,2,2004,6,22,751
3,0,0,0,0,0,2004.0,2.0,7.0,0,0,0,0,0,0,48.20876,8,11,0,4,2,2004,2,7,17
4,15,1,0,0,7,2006.0,2.0,28.0,0,0,0,1,1,70,40.00000,8,11,1,0,2,2003,7,4,4


In [ ]:
df.Income = df.Income.astype('int64')
df.Education_Score = df.Education_Score.astype('float64')

In [ ]:
from sklearn.model_selection import train_test_split
train = df[:train_df.shape[0]]
label = y_train
test = df[train_df.shape[0]:]
X_train,X_val,y_train,y_val = train_test_split(train,y_train,test_size=0.3,random_state = 1994,stratify=y_train)

In [ ]:
import lightgbm as gbm 
from lightgbm import *
m=LGBMClassifier(n_estimators=1500,random_state=1994,learning_rate=0.03,reg_alpha=0.2,colsample_bytree=0.5,bagging_fraction=0.9)
# m=RidgeCV(cv=4)
m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val.values)],eval_metric='auc', early_stopping_rounds=100,verbose=200)
p=m.predict_proba(X_val)[:,-1]

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
print(roc_auc_score(y_val,p))

Training until validation scores don't improve for 100 rounds.
[200]	training's binary_logloss: 0.398499	training's auc: 0.865976	valid_1's binary_logloss: 0.408856	valid_1's auc: 0.85526
[400]	training's binary_logloss: 0.383404	training's auc: 0.877529	valid_1's binary_logloss: 0.403631	valid_1's auc: 0.858941
[600]	training's binary_logloss: 0.371912	training's auc: 0.886284	valid_1's binary_logloss: 0.401036	valid_1's auc: 0.860899
[800]	training's binary_logloss: 0.362115	training's auc: 0.893354	valid_1's binary_logloss: 0.399442	valid_1's auc: 0.86207
[1000]	training's binary_logloss: 0.353049	training's auc: 0.89972	valid_1's binary_logloss: 0.398295	valid_1's auc: 0.863018
[1200]	training's binary_logloss: 0.344944	training's auc: 0.905186	valid_1's binary_logloss: 0.397284	valid_1's auc: 0.863734
[1400]	training's binary_logloss: 0.337338	training's auc: 0.91026	valid_1's binary_logloss: 0.396402	valid_1's auc: 0.864407
Did not meet early stopping. Best iteration is:
[1500]	t

In [ ]:
m.feature_importances_

array([2008,  686,    0,  243, 1092, 1447, 4358, 4957,  215,  379,  232,
        231, 1655, 2705, 3915, 2946, 1950,  887, 2013,   93, 1694, 3424,
       4161, 3709])

In [ ]:
confusion_matrix(y_val,p>0.5)

array([[14948,  1476],
       [ 2412,  3748]])

In [ ]:
sorted(zip(m.feature_importances_,X_train),reverse=True)

[(4957, 'Reg_day'),
 (4358, 'Reg_month'),
 (4161, 'fi_day'),
 (3915, 'Age'),
 (3709, 'camp_days'),
 (3424, 'fi_month'),
 (2946, 'City_Type'),
 (2705, 'Education_Score'),
 (2013, 'Category2'),
 (2008, 'Var1'),
 (1950, 'Employer_Category'),
 (1694, 'fi_year'),
 (1655, 'Income'),
 (1447, 'Reg_year'),
 (1092, 'Var5'),
 (887, 'Category1'),
 (686, 'Var2'),
 (379, 'LinkedIn_Shared'),
 (243, 'Var4'),
 (232, 'Twitter_Shared'),
 (231, 'Facebook_Shared'),
 (215, 'Online_Follower'),
 (93, 'Category3'),
 (0, 'Var3')]

In [ ]:
err=[]
y_pred_tot=[]

feature_importance_df = pd.DataFrame()
X = df[:train_df.shape[0]]
y = label

from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.03,reg_alpha=0.2,colsample_bytree=0.5)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    
    preds=m.predict_proba(X_test,num_iteration=m.best_iteration_)[:,-1]
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = X_train.columns
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    i=i+1
    y_pred_tot.append(p)

Training until validation scores don't improve for 200 rounds.
[200]	training's binary_logloss: 0.400721	training's auc: 0.86359	valid_1's binary_logloss: 0.406856	valid_1's auc: 0.858657
[400]	training's binary_logloss: 0.386117	training's auc: 0.874723	valid_1's binary_logloss: 0.401007	valid_1's auc: 0.862679
[600]	training's binary_logloss: 0.375468	training's auc: 0.882781	valid_1's binary_logloss: 0.398043	valid_1's auc: 0.865038
[800]	training's binary_logloss: 0.36635	training's auc: 0.889642	valid_1's binary_logloss: 0.395895	valid_1's auc: 0.866788
[1000]	training's binary_logloss: 0.35829	training's auc: 0.895445	valid_1's binary_logloss: 0.394514	valid_1's auc: 0.867857
[1200]	training's binary_logloss: 0.35135	training's auc: 0.90025	valid_1's binary_logloss: 0.393725	valid_1's auc: 0.868463
[1400]	training's binary_logloss: 0.344989	training's auc: 0.90458	valid_1's binary_logloss: 0.393276	valid_1's auc: 0.868907
[1600]	training's binary_logloss: 0.338448	training's auc:

In [ ]:
np.mean(err,0)

0.8698448366490517

In [ ]:
all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[0:150]['feature'])
all_features[0:150]

,feature,importance
0,Reg_day,9388.2
1,fi_day,8027.5
2,Age,7144.9
3,Reg_month,6975.9
4,fi_month,6419.9
5,camp_days,5965.0
6,City_Type,5370.9
7,Education_Score,4773.5
8,Var1,3363.4
9,Employer_Category,3355.9


In [ ]:
df1 = X[important_features]
corr_matrix = df1.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
high_cor = [column for column in upper.columns if any(upper[column] > 0.98)]
print(len(high_cor))
print(high_cor)

0
[]


In [ ]:
sub_df = pd.DataFrame(test_df)
sub_df["Outcome"] = np.mean(y_pred_tot,0)
sub_final_df = test_df[['Patient_ID','Health_Camp_ID','Outcome']]
sub_final_df.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.636203
1,500633,6584,0.535180
2,506945,6582,0.110546
3,497447,6551,0.685534
4,496446,6533,0.035902


In [ ]:
sub_final_df.to_csv("submission.csv", index=False)

In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>